# Imports

In [2]:
import tensorflow as tf
import os
import matplotlib as mp
import numpy as np
import matplotlib.pyplot as plt

from stereonet.generator import TrainingGeneratorStereoNet
from refocus_algorithms.layered_dof_tf import layered_bluring
from blur_baseline.model import refnet_blur_baseline
from tf_utils import optimistic_restore
from tf_gif_summary import gif_summary

# Parameters and Flags

In [8]:
# path to save
full_w = 960
full_h = 540
resolution = (full_h,full_w)

# checkpoint to load
#start from scratch with None
stereonet_weigts = None#'/home/matthieu/kgx_nfs2/ben/stereo_net/checkpoints/stereoNet_testrun_5.15377520732e-07_step_100000val_0.ckpt'
#resume training
checkpoint_to_load = None#''
#lr = 1.09418989132e-06
#step = 42000
is_from_beginning = True

n_batch = 1
n_views = 1

target_disparity = [0, 50, 100, 150, 200, 250]
blur_magnitude = 0.1
max_disp = 300
min_disp = 0
downsampling_trick_max_kernel_size=11

#
logdir = "./logs_aperture_suppervision"
checkpointdir = "./checkpoints_aperture_suppervision"

# Parsing files

In [ ]:
n_every = 1
sanity_check=True
path_to_left_rgb = '/home/matthieu/kgx_nfs2/data/external/sceneflow/frames_cleanpass/35mm_focallength/scene_mixed_train/left/'
path_to_right_rgb = '/home/matthieu/kgx_nfs2/data/external/sceneflow/frames_cleanpass/35mm_focallength/scene_mixed_train/right'
left_rgb_names_train = [os.path.join(path_to_left_rgb,each_name) for each_name in os.listdir(path_to_left_rgb)][0::n_every]
right_rgb_names_train = [os.path.join(path_to_right_rgb,each_name) for each_name in os.listdir(path_to_right_rgb)][0::n_every]

path_to_disparity = '/home/matthieu/kgx_nfs2/data/external/sceneflow/disparity/35mm_focallength/scene_mixed_train'
disparity_names_train = [os.path.join(path_to_disparity,each_name) for each_name in os.listdir(path_to_disparity)][0::n_every]

left_rgb_names_train.sort()
right_rgb_names_train.sort()
disparity_names_train.sort()

print("sanity check started.")
n_train_data = len(left_rgb_names_train)

assert len(left_rgb_names_train) == len(right_rgb_names_train) == len(disparity_names_train), "Error : number of files doesn't match"

for idx in range(n_train_data):
    left_rgb = left_rgb_names_train[idx]
    right_rgb = right_rgb_names_train[idx]
    disparity = disparity_names_train[idx]

    assert left_rgb.split('/')[-1].split('.')[-2] == right_rgb.split('/')[-1].split('.')[-2] == disparity.split('/')[-1].split('.')[-2],"Error : wrong file name match "+left_rgb+ " "+right_rgb+ " "+disparity

    if idx % 500 == 0:
        print(idx,"out of",n_train_data,"completed")

print("sanity check finished.")

# Setting up the inputs

In [ ]:
tf.reset_default_graph()#
# training generator
training_generator = iter(TrainingGeneratorStereoNet(left_rgb_names_train, right_rgb_names_train, disparity_names_train))
generator_data_type = (tf.float32, tf.float32, tf.float32)
training_set = tf.data.Dataset.from_generator(lambda: training_generator, generator_data_type)
training_set = training_set.batch(n_batch)
buffer_size = 1
training_set = training_set.prefetch(buffer_size)
# get training iterators
training_iterator = training_set.make_initializable_iterator()
# get data placeholders
rgb_left_train, rgb_right_train, disparity_train = training_iterator.get_next()

# Sizes at compile time
rgb_left_train.set_shape(tf.TensorShape([n_batch, full_h, full_w, 3]))
rgb_right_train.set_shape(tf.TensorShape([n_batch, full_h, full_w, 3]))
disparity_train.set_shape(tf.TensorShape([n_batch, full_h, full_w, 1]))


# Network and Outputs

In [5]:
#Setting up the stereonet
tf.logging.info("Network Declaration:")
#FIXME:can input several disparity plane 
blur_image, disparity, stereonet_intermediate_steps = refnet_blur_baseline(rgb_left_train, rgb_right_train, 
                                                       target_disparity, blur_magnitude,
                                                       downsampling_trick_max_kernel_size=downsampling_trick_max_kernel_size,
                                                       min_disp = min_disp, max_disp = max_disp,
                                                       is_training = True,  from_stage = "disparity_map", 
                                                       stop_grads = False, differenciable=True)
blur_image = tf.concat(blur_image, 0)
#Get disparity blur image
tf.logging.info("GT rebluring:")
blur_image_gt = [layered_bluring(rgb_left_train, disparity_train, 
                                f,blur_magnitude, 
                                min_disp = min_disp, max_disp = max_disp,
                                downsampling_trick_max_kernel_size=downsampling_trick_max_kernel_size,
                                differenciable=False) for f in target_disparity]
blur_image_gt = tf.concat(blur_image_gt, 0)

#FIXME: change stupid loss, barron? combine with stereo ? also use several focus planes?
#loss on image
def l2(x,y, mask=None):
    if mask is None:
        return tf.reduce_mean(tf.pow(x-y,2))
    else:
        return tf.reduce_sum(mask*tf.pow(x-y,2))/tf.reduce_sum(mask)

def barron_spec(x):    
    return tf.reduce_mean(tf.sqrt(tf.square(x/2.0)+1.)-1.)


#aperure suppervision
valid = tf.to_float(disparity_train>= min_disp)*tf.to_float(disparity_train<= max_disp)
loss_blur = l2(blur_image,blur_image_gt, valid)

disparity_map_1_2 = stereonet_intermediate_steps["disparity_map_1_2"]
disparity_map_1_4 = stereonet_intermediate_steps["disparity_map_1_4"]
disparity_map_1_8 = stereonet_intermediate_steps["disparity_map_1_8"]

# =============================================================================
# #depth  supervision
# loss_disparity_1_1  =  l2(disparity, disparity_train, valid)/(max_disp-min_disp)
# disparity_map_1_2 = stereonet_intermediate_steps["disparity_map_1_2"]
# disparity_map_1_4 = stereonet_intermediate_steps["disparity_map_1_4"]
# disparity_map_1_8 = stereonet_intermediate_steps["disparity_map_1_8"]
# disparity_train_1_2 = tf.image.resize_bilinear(disparity_train, disparity_map_1_2.shape[1:3])
# disparity_train_1_4 = tf.image.resize_bilinear(disparity_train, disparity_map_1_4.shape[1:3])
# disparity_train_1_8 = tf.image.resize_bilinear(disparity_train, disparity_map_1_8.shape[1:3])
# valid_1_2 = tf.image.resize_bilinear(valid, disparity_map_1_2.shape[1:3])
# valid_1_4 = tf.image.resize_bilinear(valid, disparity_map_1_4.shape[1:3])
# valid_1_8 = tf.image.resize_bilinear(valid, disparity_map_1_8.shape[1:3])
# loss_disparity_1_2  =  l2(disparity_map_1_2, disparity_train_1_2, valid_1_2)/(0.5*(max_disp-min_disp))
# loss_disparity_1_4  =  l2(disparity_map_1_4, disparity_train_1_4, valid_1_4)/(0.25*(max_disp-min_disp))
# loss_disparity_1_8  =  l2(disparity_map_1_8, disparity_train_1_8, valid_1_8)/(0.125*(max_disp-min_disp))
# loss_disparity = loss_disparity_1_1+0.5*loss_disparity_1_2+0.25*loss_disparity_1_4+0.125*loss_disparity_1_8
# loss = loss_blur+loss_disparity
# loss=tf.check_numerics(loss, "Nan in loss")
# =============================================================================

######Stereo supervision multiscale only######
# def _reconstruct_with_disparity(img_to_move,disparity):
#     """
#     Lef-consistancy check with hard constraint for training
#     """
#     batch_size = int(img_to_move.shape[0])
#     height  = int(img_to_move.shape[1])
#     width = int(img_to_move.shape[2])
#     b = np.arange(0,batch_size)
#     x = np.arange(0,width)
#     y = np.arange(0,height)
#     yv,bv,xv = np.meshgrid(y,b,x)
#     xv_with_disparity = tf.convert_to_tensor(np.expand_dims(xv,axis=3),tf.float32) - disparity
#     xv_with_disparity = tf.clip_by_value(xv_with_disparity, 0, width)
#     bv = np.expand_dims(bv,axis=3)
#     yv = np.expand_dims(yv,axis=3)
#     bv_tensor = tf.convert_to_tensor(bv,tf.int32)
#     yv_tensor = tf.convert_to_tensor(yv,tf.int32)
#     recon_grid_x_floor = tf.floor(xv_with_disparity, name='recon_floor')
#     coeff_for_floor = 1 - tf.abs(xv_with_disparity - recon_grid_x_floor)
#     recon_value_floor = tf.gather_nd(img_to_move,tf.concat([bv_tensor,yv_tensor,tf.cast(recon_grid_x_floor,tf.int32)],axis=3))
#     recon_grid_x_ceil = tf.ceil(xv_with_disparity, name='recon_ceil')
#     coeff_for_ceil = 1 - tf.abs(recon_grid_x_ceil - xv_with_disparity)
#     recon_value_ceil = tf.gather_nd(img_to_move,tf.concat([bv_tensor,yv_tensor,tf.cast(recon_grid_x_ceil,tf.int32)],axis=3))
#     recon = recon_value_floor * coeff_for_floor + recon_value_ceil * coeff_for_ceil
#     return recon
# 
# rgb_right_train_1_1 = rgb_right_train
# rgb_right_train_1_2 = tf.image.resize_bilinear(rgb_right_train, disparity_map_1_2.shape[1:3])
# rgb_right_train_1_4 = tf.image.resize_bilinear(rgb_right_train, disparity_map_1_4.shape[1:3])
# rgb_right_train_1_8 = tf.image.resize_bilinear(rgb_right_train, disparity_map_1_8.shape[1:3])
# rgb_left_train_1_1 = rgb_left_train
# rgb_left_train_1_2 = tf.image.resize_bilinear(rgb_left_train, disparity_map_1_2.shape[1:3])
# rgb_left_train_1_4 = tf.image.resize_bilinear(rgb_left_train, disparity_map_1_4.shape[1:3])
# rgb_left_train_1_8 = tf.image.resize_bilinear(rgb_left_train, disparity_map_1_8.shape[1:3])
# 
# warped_right_1_1 = _reconstruct_with_disparity(rgb_right_train_1_1, disparity)
# warped_right_1_2 = _reconstruct_with_disparity(rgb_right_train_1_2, disparity_map_1_2)
# warped_right_1_4 = _reconstruct_with_disparity(rgb_right_train_1_4, disparity_map_1_4)
# warped_right_1_8 = _reconstruct_with_disparity(rgb_right_train_1_8, disparity_map_1_8)
# 
# loss_stereo_1_1 = l2(warped_right_1_1,rgb_left_train_1_1)
# loss_stereo_1_2 = l2(warped_right_1_2,rgb_left_train_1_2)
# loss_stereo_1_4 = l2(warped_right_1_4,rgb_left_train_1_4)
# loss_stereo_1_8 = l2(warped_right_1_8,rgb_left_train_1_8)
# 
# loss_stereo = (8/15.0)*loss_stereo_1_1+(4/15.0)*loss_stereo_1_2+(2/15.0)*loss_stereo_1_4+(1/15.0)*loss_stereo_1_8
# ######

loss = loss_blur# loss_stereo
loss=tf.check_numerics(loss, "Nan in loss")


#definition learning step
learning_rate = tf.placeholder(tf.float32)
optimizer = tf.train.RMSPropOptimizer(learning_rate = learning_rate)
train_op = optimizer.minimize(loss=loss)



INFO:tensorflow:Network Declaration:
INFO:tensorflow:Refocusing with disparity range 0.000000 300.000000
INFO:tensorflow:Disparity slice 0.000000
INFO:tensorflow:Bluring with radius size 0.000000 kernel size 3
INFO:tensorflow:Bluring with radius size 0.000000 kernel size 3
INFO:tensorflow:Disparity slice 10.000000
INFO:tensorflow:Bluring with radius size 1.000000 kernel size 5
INFO:tensorflow:Bluring with radius size 1.000000 kernel size 5
INFO:tensorflow:Disparity slice 20.000000
INFO:tensorflow:Bluring with radius size 2.000000 kernel size 7
INFO:tensorflow:Bluring with radius size 2.000000 kernel size 7
INFO:tensorflow:Disparity slice 30.000000
INFO:tensorflow:Bluring with radius size 3.000000 kernel size 9
INFO:tensorflow:Bluring with radius size 3.000000 kernel size 9
INFO:tensorflow:Disparity slice 40.000000
INFO:tensorflow:Bluring with radius size 4.000000 kernel size 11
INFO:tensorflow:Bluring with radius size 4.000000 kernel size 11
INFO:tensorflow:Disparity slice 50.000000
IN

INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Bluring with radius size 26.000000 kernel size 55
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Disparity slice 270.000000
INFO:tensorflow:Bluring with radius size 27.000000 kernel size 57
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Bluring with radius size 27.000000 kernel size 57
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Disparity slice 280.000000
INFO:tensorflow:Bluring with radius size 28.000000 kernel size 59
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Bluring with radius size 28.000000 kernel size 59
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Disparity slice 290.000000
INFO:tensorflow:Bluring with radius s

INFO:tensorflow:Upsampling
INFO:tensorflow:Bluring with radius size 19.000000 kernel size 41
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Disparity slice 250.000000
INFO:tensorflow:Bluring with radius size 20.000000 kernel size 43
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Bluring with radius size 20.000000 kernel size 43
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Disparity slice 260.000000
INFO:tensorflow:Bluring with radius size 21.000000 kernel size 45
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Bluring with radius size 21.000000 kernel size 45
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Disparity slice 270.000000
INFO:tensorflow:Bluring with radius size 22.000000 kernel size 47
INFO:tensorflow:Downsampling

INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Disparity slice 230.000000
INFO:tensorflow:Bluring with radius size 13.000000 kernel size 29
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Bluring with radius size 13.000000 kernel size 29
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Disparity slice 240.000000
INFO:tensorflow:Bluring with radius size 14.000000 kernel size 31
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Bluring with radius size 14.000000 kernel size 31
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Disparity slice 250.000000
INFO:tensorflow:Bluring with radius size 15.000000 kernel size 33
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Bluring with radius s

INFO:tensorflow:Upsampling
INFO:tensorflow:Disparity slice 210.000000
INFO:tensorflow:Bluring with radius size 6.000000 kernel size 15
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Bluring with radius size 6.000000 kernel size 15
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Disparity slice 220.000000
INFO:tensorflow:Bluring with radius size 7.000000 kernel size 17
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Bluring with radius size 7.000000 kernel size 17
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Disparity slice 230.000000
INFO:tensorflow:Bluring with radius size 8.000000 kernel size 19
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Bluring with radius size 8.000000 kernel size 19
INFO:tensorflow:Downsampling to ma

KeyboardInterrupt: 

# Initialisaiton

In [ ]:
# =============================================================================
# Initializors
# =============================================================================
tf.logging.info("Initialisaiton")

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

writer = tf.summary.FileWriter(logdir, sess.graph)

init = tf.global_variables_initializer()
sess.run(init)
sess.run(training_iterator.initializer)
train_init_op = training_iterator.make_initializer(training_set)
sess.run(train_init_op)
#img_left = sess.run(rgb_left_train)
#print img_left.shape

# =============================================================================
# Checkpoints and Visualizations
# =============================================================================
if is_from_beginning:
    lr = 1e-5
    step = 0
    val_step = 0
    losses = np.zeros((1,3))
    #loading pretrained stereonet weigts
    if stereonet_weigts is not None:
        tf.logging.info("Restoring variables from "+stereonet_weigts)
        rv = optimistic_restore(sess, stereonet_weigts)
        tf.logging.info("Restored %d vars"%rv)
else:
    saver1 = tf.train.Saver(var_list=tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES))
    saver1.restore(sess, checkpoint_to_load)
    val_step = 0
    
    

In [ ]:

#tf.summary.scalar('loss_blur', loss_blur)
#tf.summary.scalar('loss_stereo', loss_stereo)
tf.summary.scalar('loss', loss)

conf_volume = tf.nn.softmax(-stereonet_intermediate_steps["cost_volume_left_view"], dim=-1)
gif_summary('conf_volume', tf.cast(255*tf.expand_dims(tf.transpose(conf_volume, [0,3,1,2]), -1), tf.uint8), fps=10)


tf.summary.image('GT_blur', tf.clip_by_value(blur_image_gt,0,1))
tf.summary.image('blur_image', tf.clip_by_value(blur_image,0,1))
tf.summary.image('diff_image', tf.clip_by_value(tf.abs(blur_image-blur_image_gt),0,1))

tf.summary.image('disparity_1_2', disparity_map_1_2)
tf.summary.image('disparity_1_4', disparity_map_1_4)
tf.summary.image('disparity_1_8', disparity_map_1_8)

tf.summary.image('disparity', disparity)
tf.summary.image('GT_disparity', disparity_train)

tf.summary.histogram( 'disparity_histogram', disparity)

train_summary_op = tf.summary.merge_all()

# Training

In [ ]:
n_epoch = 100
n_iter = 200000
save_n_every_step = 2000
show_n_every_step = 1000

force_show_steps = [2,4,8,16,32,64,128,256,512]

loss_values=[]

# =============================================================================
# Training loop
# =============================================================================
tf.logging.info("Starting training")

for each_epoch in range(n_epoch):
    for each_iter in range(n_iter):
        
        run_list = [train_op,
                    train_summary_op,
                    loss
                   ]
        
        output = sess.run(run_list, feed_dict={learning_rate:lr})
        
        train_summary = output[1]
        loss_out = output[2]

        if step != 0 and step % save_n_every_step == 0:
            lr *= 0.9
            saver = tf.train.Saver(var_list = tf.global_variables())
            save_path = saver.save(sess,checkpointdir+"/run_" + str(lr) + "_step_" + str(step) + "val_" + str(val_step) + '.ckpt')
        
        if (step != 0 and step % show_n_every_step == 0) or step in force_show_steps:
            tf.logging.info("Step %d. Loss=%f"%(step,loss_out))
            writer.add_summary(train_summary,int(step))
            
        else:
            from math import isnan
            if isnan(loss_out):
                raise BaseException("Loss in Nan at step %d"%step)
       
        step += 1

# Testing (aperture and depth supervision)

In [5]:
from PIL import Image
from stereonet.utils import readPFM

import tensorflow as tf
import os
import numpy as np
from refocus_algorithms.layered_dof_tf import layered_bluring
from blur_baseline.model import refnet_blur_baseline

checkpoint_aperture = "/home/matthieu/dev/refnet/blur_baseline/checkpoints_aperture_suppervision/run_1.09418989132e-06_step_42000val_0.ckpt"
checkpoint_depth  = '/home/matthieu/kgx_nfs2/ben/stereo_net_refactored/checkpts/sceneFlow/stereoNet_ScenFlow_driving_1.7888498861e-07_step_120000.ckpt'




#params
target_disparity = [0, 50, 100, 150, 200, 250]
blur_magnitude = 0.1
max_disp = 300
min_disp = 0
downsampling_trick_max_kernel_size=11
aperture_suppervision = False
#
full_w = 960
full_h = 540
resolution = (full_h,full_w)
#
out_folder  = "/home/matthieu/dev/refnet/blur_baseline/test_out"
#
if aperture_suppervision:
    checkpoint_to_load = checkpoint_aperture
else:
    checkpoint_to_load = checkpoint_depth      


In [6]:
tf.reset_default_graph()

#inputs
path_to_left_rgb = '/home/matthieu/kgx_nfs2/data/external/sceneflow/frames_cleanpass/35mm_focallength/scene_mixed_test/left/'
path_to_right_rgb = '/home/matthieu/kgx_nfs2/data/external/sceneflow/frames_cleanpass/35mm_focallength/scene_mixed_test/right'
left_rgb_names_test = [os.path.join(path_to_left_rgb,each_name) for each_name in os.listdir(path_to_left_rgb)]
right_rgb_names_test = [os.path.join(path_to_right_rgb,each_name) for each_name in os.listdir(path_to_right_rgb)]
path_to_disparity = '/home/matthieu/kgx_nfs2/data/external/sceneflow/disparity/35mm_focallength/scene_mixed_test'
disparity_names_test = [os.path.join(path_to_disparity,each_name) for each_name in os.listdir(path_to_disparity)]
left_rgb_names_test.sort()
right_rgb_names_test.sort()
disparity_names_test.sort()

# Sizes at compile time
rgb_left_test = tf.placeholder(shape=[1, full_h, full_w, 3], dtype=tf.float32)
rgb_right_test = tf.placeholder(shape=[1, full_h, full_w, 3], dtype=tf.float32)
disparity_test = tf.placeholder(shape=[1, full_h, full_w, 1], dtype=tf.float32)

#Get forward 
blur_image, disparity, stereonet_intermediate_steps = refnet_blur_baseline(rgb_left_test, rgb_right_test, 
                                                       target_disparity, blur_magnitude,
                                                       downsampling_trick_max_kernel_size=downsampling_trick_max_kernel_size,
                                                       min_disp = min_disp, max_disp = max_disp,
                                                       is_training = True,  from_stage = "disparity_map", 
                                                       stop_grads = False, differenciable=True)
blur_image = tf.concat(blur_image, 0)
#Get disparity blur image
blur_image_gt = [layered_bluring(rgb_left_test, disparity_test, 
                                f,blur_magnitude, 
                                min_disp = min_disp, max_disp = max_disp,
                                downsampling_trick_max_kernel_size=downsampling_trick_max_kernel_size,
                                differenciable=False) for f in target_disparity]
blur_image_gt = tf.concat(blur_image_gt, 0)


INFO:tensorflow:Refocusing with disparity range 0.000000 300.000000
INFO:tensorflow:Disparity slice 0.000000
INFO:tensorflow:Bluring with radius size 0.000000 kernel size 3
INFO:tensorflow:Bluring with radius size 0.000000 kernel size 3
INFO:tensorflow:Disparity slice 10.000000
INFO:tensorflow:Bluring with radius size 1.000000 kernel size 5
INFO:tensorflow:Bluring with radius size 1.000000 kernel size 5
INFO:tensorflow:Disparity slice 20.000000
INFO:tensorflow:Bluring with radius size 2.000000 kernel size 7
INFO:tensorflow:Bluring with radius size 2.000000 kernel size 7
INFO:tensorflow:Disparity slice 30.000000
INFO:tensorflow:Bluring with radius size 3.000000 kernel size 9
INFO:tensorflow:Bluring with radius size 3.000000 kernel size 9
INFO:tensorflow:Disparity slice 40.000000
INFO:tensorflow:Bluring with radius size 4.000000 kernel size 11
INFO:tensorflow:Bluring with radius size 4.000000 kernel size 11
INFO:tensorflow:Disparity slice 50.000000
INFO:tensorflow:Bluring with radius siz

INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Bluring with radius size 26.000000 kernel size 55
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Disparity slice 270.000000
INFO:tensorflow:Bluring with radius size 27.000000 kernel size 57
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Bluring with radius size 27.000000 kernel size 57
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Disparity slice 280.000000
INFO:tensorflow:Bluring with radius size 28.000000 kernel size 59
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Bluring with radius size 28.000000 kernel size 59
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Disparity slice 290.000000
INFO:tensorflow:Bluring with radius s

INFO:tensorflow:Upsampling
INFO:tensorflow:Bluring with radius size 19.000000 kernel size 41
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Disparity slice 250.000000
INFO:tensorflow:Bluring with radius size 20.000000 kernel size 43
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Bluring with radius size 20.000000 kernel size 43
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Disparity slice 260.000000
INFO:tensorflow:Bluring with radius size 21.000000 kernel size 45
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Bluring with radius size 21.000000 kernel size 45
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Disparity slice 270.000000
INFO:tensorflow:Bluring with radius size 22.000000 kernel size 47
INFO:tensorflow:Downsampling

INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Disparity slice 230.000000
INFO:tensorflow:Bluring with radius size 13.000000 kernel size 29
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Bluring with radius size 13.000000 kernel size 29
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Disparity slice 240.000000
INFO:tensorflow:Bluring with radius size 14.000000 kernel size 31
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Bluring with radius size 14.000000 kernel size 31
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Disparity slice 250.000000
INFO:tensorflow:Bluring with radius size 15.000000 kernel size 33
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Bluring with radius s

INFO:tensorflow:Upsampling
INFO:tensorflow:Disparity slice 210.000000
INFO:tensorflow:Bluring with radius size 6.000000 kernel size 15
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Bluring with radius size 6.000000 kernel size 15
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Disparity slice 220.000000
INFO:tensorflow:Bluring with radius size 7.000000 kernel size 17
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Bluring with radius size 7.000000 kernel size 17
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Disparity slice 230.000000
INFO:tensorflow:Bluring with radius size 8.000000 kernel size 19
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Bluring with radius size 8.000000 kernel size 19
INFO:tensorflow:Downsampling to ma

...

INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Disparity slice 60.000000
INFO:tensorflow:Bluring with radius size 4.000000 kernel size 11
INFO:tensorflow:Bluring with radius size 4.000000 kernel size 11
INFO:tensorflow:Disparity slice 70.000000
INFO:tensorflow:Bluring with radius size 3.000000 kernel size 9
INFO:tensorflow:Bluring with radius size 3.000000 kernel size 9
INFO:tensorflow:Disparity slice 80.000000
INFO:tensorflow:Bluring with radius size 2.000000 kernel size 7
INFO:tensorflow:Bluring with radius size 2.000000 kernel size 7
INFO:tensorflow:Disparity slice 90.000000
INFO:tensorflow:Bluring with radius size 1.000000 kernel size 5
INFO:tensorflow:Bluring with radius size 1.000000 kernel size 5
INFO:tensorflow:Disparity slice 100.000000
INFO:tensorflow:Bluring with radius size 0.000000 kernel size 3
INFO:tensorflow:Bluring with radius size 0.000000 kernel size 3
INFO:tensorflow:Disparity slice 110.000000
INFO:tensorflow:Blur

INFO:tensorflow:Upsampling
INFO:tensorflow:Disparity slice 40.000000
INFO:tensorflow:Bluring with radius size 11.000000 kernel size 25
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Bluring with radius size 11.000000 kernel size 25
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Disparity slice 50.000000
INFO:tensorflow:Bluring with radius size 10.000000 kernel size 23
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Bluring with radius size 10.000000 kernel size 23
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Disparity slice 60.000000
INFO:tensorflow:Bluring with radius size 9.000000 kernel size 21
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Bluring with radius size 9.000000 kernel size 21
INFO:tensorflow:Downsampling to m

INFO:tensorflow:Bluring with radius size 18.000000 kernel size 39
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Bluring with radius size 18.000000 kernel size 39
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Disparity slice 30.000000
INFO:tensorflow:Bluring with radius size 17.000000 kernel size 37
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Bluring with radius size 17.000000 kernel size 37
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Disparity slice 40.000000
INFO:tensorflow:Bluring with radius size 16.000000 kernel size 35
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Bluring with radius size 16.000000 kernel size 35
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:

INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Bluring with radius size 25.000000 kernel size 53
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Disparity slice 10.000000
INFO:tensorflow:Bluring with radius size 24.000000 kernel size 51
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Bluring with radius size 24.000000 kernel size 51
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Disparity slice 20.000000
INFO:tensorflow:Bluring with radius size 23.000000 kernel size 49
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Bluring with radius size 23.000000 kernel size 49
INFO:tensorflow:Downsampling to match max radius size 11
INFO:tensorflow:Upsampling
INFO:tensorflow:Disparity slice 30.000000
INFO:tensorflow:Bluring with radius size

INFO:tensorflow:Bluring with radius size 2.000000 kernel size 7
INFO:tensorflow:Disparity slice 280.000000
INFO:tensorflow:Bluring with radius size 3.000000 kernel size 9
INFO:tensorflow:Bluring with radius size 3.000000 kernel size 9
INFO:tensorflow:Disparity slice 290.000000
INFO:tensorflow:Bluring with radius size 4.000000 kernel size 11
INFO:tensorflow:Bluring with radius size 4.000000 kernel size 11
INFO:tensorflow:Image composition


In [7]:
#open sess
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

#init
init = tf.global_variables_initializer()
sess.run(init)

#restore weigts 
saver1 = tf.train.Saver(var_list=tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES))
saver1.restore(sess, checkpoint_to_load)

#do test
for i in range(len(left_rgb_names_test)):
    print(i)
    rgb_left_data = np.expand_dims(np.asarray(Image.open(left_rgb_names_test[i]))/255.0, 0)
    rgb_right_data = np.expand_dims(np.asarray(Image.open(right_rgb_names_test[i]))/255.0, 0)
    disparity_data = np.expand_dims(np.expand_dims(readPFM(disparity_names_test[i]), 0), -1)

    blur_GT_out = sess.run(blur_image_gt,
                          feed_dict={rgb_left_test:rgb_left_data, 
                                     disparity_test:disparity_data})
    blur_image_out, disparity_out = sess.run([blur_image, disparity], 
                                             feed_dict={ rgb_left_test:rgb_left_data,
                                                         rgb_right_test:rgb_right_data})
    if aperture_suppervision:
        Image.fromarray((255*disparity_out[0,:,:,0]/max_disp).astype(np.uint8)).save(out_folder+"/%d_disp_aperture.png"%(i))
    else:
        Image.fromarray((255*disparity_out[0,:,:,0]/max_disp).astype(np.uint8)).save(out_folder+"/%d_disp.png"%(i))
    Image.fromarray((255*disparity_data[0,:,:,0]/max_disp).astype(np.uint8)).save(out_folder+"/%d_disp_GT.png"%(i))
    for f in range(len(target_disparity)):
        Image.fromarray((255*blur_GT_out[f]).astype(np.uint8)).save(out_folder+"/%d_%d_GT.png"%(i,target_disparity[f]))
        if aperture_suppervision:
            Image.fromarray((255*blur_image_out[f]).astype(np.uint8)).save(out_folder+"/%d_%d_sequential_aperture.png"%(i,target_disparity[f]))
        else:
            Image.fromarray((255*blur_image_out[f]).astype(np.uint8)).save(out_folder+"/%d_%d_sequential.png"%(i,target_disparity[f]))

INFO:tensorflow:Restoring parameters from /home/matthieu/kgx_nfs2/ben/stereo_net_refactored/checkpts/sceneFlow/stereoNet_ScenFlow_driving_1.7888498861e-07_step_120000.ckpt
0
1
2
3
4
5
6
7
8
9
10
